In [1]:
%cd ..

In [2]:
import json
import re

In [3]:
with open("../data/recipes/amounts.json", encoding="utf-8") as f:
    amounts = json.load(f)

In [4]:
re1 = re.compile(r"^([\d½¼¾⅛]+(?: [½¼¾⅛])?(?:,\d+)?)(?: ((?:kl\. |gr\. )?\w+(?:, gestr\.|, gehäuft)?)(?:/n|/e|\(n\)|\(e\)|\.)?)?$")
re_singol = re.compile(r"^(kl. |gr. )?(Tasse|mg|ml|Tüte|Kopf|Stück|Spritzer|Teil|Becher|Blatt|Zehe|Blätter|Bund|Dose|[ET]L(,.*)?|cl|dl|Flasche|Fläschchen|Glas|Gläser|Handvoll|Körner|Liter|Msp.|Paar|Paket|Pck.|Pkt.|Port.|Prise.*|Scheibe)(/[en])?$")


In [5]:
replacings = "gestr.=gestrichen,geh.=gehäuft,m.-große=mittelgroße,m.-großer=mittelgroßer,TL=Teelöffel,Eßlöffel=Esslöffel,EL=Esslöffel,Msp.=Messerspitze,Pck.=Packung,Pkt.=Packung,Port.=Portion".split(",")

replacings = {ee.split("=")[0]: ee.split("=")[1] for ee in replacings}

In [6]:
parsed1 = []
for _amount in amounts:
    amount = _amount
    if amount == "":
        amount = "1"
    #amount = amount.replace("m.-große", "mittelgroße")
    if re_singol.match(amount):
        amount = "1 " + amount
    for bef, aft in replacings.items():
        amount = re.sub(r"\b" + re.escape(bef) + r"(\b|$)", aft, amount)
    m = re1.match(amount)
    if m is not None:
        parsed1.append((_amount, *m.groups()))
    else:
        print("UNMATCHED  ", _amount, amount)
        parsed1.append((_amount, None, None))

UNMATCHED   0,2 TL, gestr. 0,2 Teelöffel, gestrichen
UNMATCHED   0,3 TL, gestr. 0,3 Teelöffel, gestrichen
UNMATCHED   0,33 EL, gestr. 0,33 Esslöffel, gestrichen
UNMATCHED   0,33 TL, gestr. 0,33 Teelöffel, gestrichen
UNMATCHED   1 EL, gestr. 1 Esslöffel, gestrichen
UNMATCHED   1 TL, gestr. 1 Teelöffel, gestrichen
UNMATCHED   1 einige Stiele 1 einige Stiele
UNMATCHED   1 m.-großes 1 m.-großes
UNMATCHED   1 n. B. 1 n. B.
UNMATCHED   1 ½ EL, gestr. 1 ½ Esslöffel, gestrichen
UNMATCHED   1 ½ TL, gestr. 1 ½ Teelöffel, gestrichen
UNMATCHED   10 EL, gestr. 10 Esslöffel, gestrichen
UNMATCHED   10 TL, gestr. 10 Teelöffel, gestrichen
UNMATCHED   10 einige Stiele 10 einige Stiele
UNMATCHED   10 n. B. 10 n. B.
UNMATCHED   12 TL, gestr. 12 Teelöffel, gestrichen
UNMATCHED   125 n. B. 125 n. B.
UNMATCHED   14 TL, gestr. 14 Teelöffel, gestrichen
UNMATCHED   15 EL, gestr. 15 Esslöffel, gestrichen
UNMATCHED   18 EL, gestr. 18 Esslöffel, gestrichen
UNMATCHED   2 EL, gestr. 2 Esslöffel, gestrichen
UNMATCHED

In [7]:
len(parsed1), len(amounts)

(2782, 2782)

In [8]:
parsed1[100]

('0,33 Tasse', '0,33', 'Tasse')

In [9]:
re_num = re.compile(r"([\d½¼¾⅛]+)(?: ([½¼¾⅛]))?(?:,(\d+))?")

In [10]:
def parse_num(amount, unit):
    if amount is None:
        return (None, None)
    fs = {"½": 1/2, "¼": 1/4, "¾": 3/4, "⅛": 1/8}
    whole, addition, fraction = re_num.match(amount).groups()
    whole = fs[whole] if whole in fs else int(whole)
    addition = fs[addition] if addition is not None else 0
    fraction = float("0." + fraction) if fraction is not None else 0
    total_amount = whole + addition + fraction
    if unit == "kg":
        unit = "g"
        total_amount *= 1000
    if unit == "mg":
        unit = "g"
        total_amount /= 1000
    if unit and (unit.lower() == "l" or unit.lower() == "liter"):
        unit = "ml"
        total_amount *= 1000
    if unit == "cl":
        unit = "ml"
        total_amount *= 10
    return total_amount, unit

parsed_amounts = {orig: parse_num(n, e) for orig, n, e in parsed1}

In [11]:
parsed_amounts

{'': (1, None),
 '0,01 g': (0.01, 'g'),
 '0,02 kg': (20.0, 'g'),
 '0,03 kg': (30.0, 'g'),
 '0,03 ml': (0.03, 'ml'),
 '0,04 Liter': (40.0, 'ml'),
 '0,04 kg': (40.0, 'g'),
 '0,05 Liter': (50.0, 'ml'),
 '0,05 cl': (0.5, 'ml'),
 '0,05 g': (0.05, 'g'),
 '0,05 kg': (50.0, 'g'),
 '0,06 Liter': (60.0, 'ml'),
 '0,07 Liter': (70.0, 'ml'),
 '0,08 EL': (0.08, 'Esslöffel'),
 '0,08 kg': (80.0, 'g'),
 '0,1': (0.1, None),
 '0,1 Liter': (100.0, 'ml'),
 '0,1 TL': (0.1, 'Teelöffel'),
 '0,1 g': (0.1, 'g'),
 '0,1 kg': (100.0, 'g'),
 '0,1 ml': (0.1, 'ml'),
 '0,12': (0.12, None),
 '0,12 Liter': (120.0, 'ml'),
 '0,12 TL': (0.12, 'Teelöffel'),
 '0,12 g': (0.12, 'g'),
 '0,13 kg': (130.0, 'g'),
 '0,15 Liter': (150.0, 'ml'),
 '0,15 g': (0.15, 'g'),
 '0,15 kg': (150.0, 'g'),
 '0,17': (0.17, None),
 '0,18 Liter': (180.0, 'ml'),
 '0,19 Liter': (190.0, 'ml'),
 '0,2': (0.2, None),
 '0,2 Becher': (0.2, 'Becher'),
 '0,2 Bund': (0.2, 'Bund'),
 '0,2 Dose': (0.2, 'Dose'),
 '0,2 Flasche': (0.2, 'Flasche'),
 '0,2 Liter': (20

In [13]:
with open("../data/recipes/parsed_amounts.json", "w") as f:
    json.dump(parsed_amounts, f)